# Generating Artifacts using Generative AI

This notebook demonstrates how to automate the creation of industrial asset definitions in [AWS IoT SiteWise](https://aws.amazon.com/iot-sitewise/) and [Shop Floor Connectivity (SFC)](https://github.com/aws-samples/shopfloor-connectivity) configurations using generative AI. \
We will convert human-readable descriptions of the reInvent Car Factory's facilities into formatted asset definitions and data collection configurations.

### ⛳ Goal

1. **AWS IoT SiteWise Metadata Generation**: We'll use [Amazon Bedrock's Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-call.html) with Anthropic's Claude model to generate a Metadata Transfer Job Schema for AWS IoT SiteWise.
This approach allows for rapid, scalable creation of complex asset models and hierarchies.
2. **SFC Configuration Generation**: Leveraging the same AI capabilities, we'll demonstrate how to automatically generate configurations for the Shopfloor Connectivity framework.

By the end of this notebook, you'll have a practical understanding of how generative AI can be applied to automate the creation of industrial asset definitions and data collection configurations. \
This approach demonstrates a powerful method for streamlining the setup and management of complex industrial data environments, potentially saving significant time and reducing errors in the process.

### 🗝️ Key concept refreshers

Before we dive into the practical application, let's quickly review the essential concepts:

1. **Asset Model**: A template defining the structure and characteristics of industrial equipment or processes.
2. **Asset**: A specific instance of an asset model, representing actual equipment or processes.
3. **Asset Hierarchy**: The organizational structure showing relationships between assets.
4. **[Metadata Transfer Job Schema](https://docs.aws.amazon.com/iot-sitewise/latest/userguide/bulk-operations-schema.html)**: Used to define assets and their relationships for bulk import.
5. **[Amazon Bedrock Tool Feature]((https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html))**: Provides specific context to the AI model for accurate task completion.
6. **AWS IoT SiteWise**: A managed service for collecting, organizing, and analyzing industrial equipment data.
7. **Generative AI**: AI capable of generating new content based on provided inputs and context.
8. **Shop Floor Connectivity**: A component that facilitates data ingestion from industrial equipment to the cloud.
9. **Configuration Files**: Specifications that define how the Shop Floor Connectivity component should collect and transmit data.



<div class="alert alert-block alert-warning">
<b>⚠️</b> Generative AI produces non-deterministic output. Your results may vary from the demonstrated examples.
</div>

## 1. Amazon Bedrock configuration and setup

In this section, we wil do some pre-work on the set-up needed to interract with Amazon Bedrock.

### 1.1 Set-up the AWS Client and configurations.

The following kernel sets up the necessary AWS client configuration by importing required libraries and retrieving regional metadata from an EC2 instance. It then establishes a connection to Amazon Bedrock service by creating a session and initializing a specific service client for further API interactions.

In [ ]:
import boto3,json
from ec2_metadata import ec2_metadata

aws_region = ec2_metadata.region
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime', region_name="us-west-2")

### 1.2 Amazon Bedrock function for tool-directed AI conversations

This function implements a structured interaction with Amazon Bedrock's Claude 3 Sonnet model, accepting a prompt, context, and tool_name as inputs to generate AI responses through the converse API. 
The function processes the model's response by extracting specific tool-related content from the nested JSON structure and returns the tool's input dictionary as the final result.

In [ ]:
def call_bedrock (prompt,content,tool_object):
    """
    Makes a directed call to Amazon Bedrock's Claude 3 Haiku model using the Converse API's in Bedrock.

    Args:
        prompt (str): The main instruction or question to send to the model
        content (str): The message content
        tool_object: The tool object for the Bedrock Converse API

    Returns:
        dict: The input parameters generated by the model for the specified tool use

    Example:
        response = call_bedrock(
            prompt="Analyze this data",
            context="Sample data content",
            tool_name="data_analyzer",
            tool_object=tool_config
        )

    Notes:
        - Uses Claude 3 Haiku model with deterministic output (temperature=0)
        - Maximum response length is set to 2000 tokens
        - Forces the model to use the specified tool through toolChoice configuration
        - Extracts the tool use input parameters from the model's response
    """
    message = {
    "role": "user",
    "content": [
        { "text": f"<content>{content}</content>" },
        { "text": prompt }
    ],
    }

    response = bedrock.converse(
        modelId="anthropic.claude-3-haiku-20240307-v1:0",
        messages=[message],
        inferenceConfig={
            "maxTokens": 2000, 
            "temperature": 0
        },
        toolConfig={
            "tools": tool_object,
            "toolChoice": {
                "tool": {
                    "name": tool_object[0]["toolSpec"]["name"] #get the name of the tool_object
                }
            }
        }
    )
    response_message = response['output']['message']
    response_content_blocks = response_message['content']
    # Gets first content block containing 'toolUse' or returns None if not found
    content_block = next((block for block in response_content_blocks if 'toolUse' in block), None)
    tool_use_block = content_block['toolUse']
    return tool_use_block['input'] # containing the genAI response

In the following code we define a function for nicely visualizing the structures of the generated JSON files. These are helper functions, you may execute the kernel, and proceed to the next step.

In [ ]:
from anytree import Node, RenderTree
from anytree.exporter import DotExporter


def create_tree(data, parent=None):
    """Recursively builds a tree structure from nested data using anytree.Node"""
    
    if isinstance(data, dict):
        for key, value in data.items():
            node = Node(f"{key}", parent=parent)
            create_tree(value, node)
    elif isinstance(data, list):
        for i, item in enumerate(data):
            node = Node(f"Item {i}", parent=parent)
            create_tree(item, node)
    else:
        Node(f"{data}", parent=parent)

def visualize_json(json_data):
    """Creates and prints an ASCII tree visualization of JSON/dict data structure"""
    # Parse JSON if it's a string, otherwise use the data as is
    if isinstance(json_data, str):
        data = json.loads(json_data)
    else:
        data = json_data

    # Create the root node
    root = Node("Root")

    # Build the tree
    create_tree(data, root)

    # Print the tree structure
    for pre, _, node in RenderTree(root):
        print(f"{pre}{node.name}")

## 2. AWS IoT SiteWise Asset Generation

In this module, we'll walk through the process of using Amazon Bedrock to automatically generate AWS IoT SiteWise assets based on industrial context. This process involves three main steps:

1. Defining a custom tool for Amazon Bedrock's Converse API that specifies the structure and requirements for SiteWise asset generation.
2. Retrieving relevant context from a previously created TIA (Totally Integrated Automation) project.
3. Using this context to prompt Amazon Bedrock to generate a schema for AWS IoT SiteWise assets.

### 2.1 Defining the AWS IoT SiteWise Asset Generation Tool for Amazon Bedrock

The following code defines a tool object for Amazon Bedrock's Converse API, specifying the structure and requirements for generating AWS IoT SiteWise Assets

In [ ]:
tool_object_sitewise_assets = [
    {
        "toolSpec": {
            "name": "sitewise_assets",
            "description": "Create AWS IoT Sitewise Assets",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "assets": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "assetExternalId": {
                                        "type": "string",
                                        "description": "prefix this assetName with eID_ and relpace blank spaces with _"
                                    },
                                    "assetName": {
                                        "type": "string",
                                        "description": "A two to three word name of the asset prefixed with GenAI",
                                    },
                                    "assetModelExternalId": {
                                        "type": "string",
                                        "description": "the appropriate model eID_ for the asset ",
                                        "enum": ["eID_General_Cell", "eID_Production_Unit", "eID_Regional_Unit", "eID_Enterprise_Unit"]
                                    }                                  
                                }
                            }
                        }
                    },
                    "required": [
                        "assets",
                    ]
                }
            }
        }
    }
]


### 2.2 Retrieve Context from the TIA Project in the previous exercise

To proceed with our asset generation, we'll first retrieve the TIA Portal metadata from the file ```tia_portal_meta.txt``` created in the previous chapter.

In [ ]:
with open("tia_portal_meta.txt", "r") as file:
    tia_meta_data = file.read()

print(tia_meta_data)

### 2.3. Prompt Amazon Bedrock to generate the AWS IoT SiteWise Assets Schema

Now that we have our TIA Portal metadata and the defined AWS IoT SiteWise Asset generation tool, we'll use Amazon Bedrock to create the JSON schema for our assets. 
This schema will be suitable for bulk import into AWS IoT SiteWise. 


In [ ]:
# The exported tia_meta_data serve as context
context = tia_meta_data       

# Define prompt for Amazon Bedrock to generate AWS IoT SiteWise asset JSON schema
prompt = """Please use the sitewise_assets tool to generate the a JSON schema,                  
to create assets for all cells for meta-data-bulk-import in AWS IoT Sitewise 
based on the content within the <content> tags"""                                               

# Call Amazon Bedrock with prompt and metadata to generate AWS IoT SiteWise assets schema
assets_schema = call_bedrock (prompt, context, tool_object_sitewise_assets)

# Pretty print the generated schema with proper JSON formatting
print(json.dumps(assets_schema, indent=4))

In [ ]:
# Visualize the JSON schema
visualize_json(assets_schema)

In [ ]:
# The schema structure includes an empty asset models dictionary.
asset_models = {"assetModels": []}
modified_schema = {**asset_models, **assets_schema}

#Write the assets schema to a JSON file
with open("genai_sitewise_assets_schema.json", "w") as file:
    json.dump(modified_schema, file, indent=4)

## 3. AWS IoT SiteWise Assets Hierarchy Generation

In this module, we'll walk through the process of using Amazon Bedrock to automatically generate the AWS IoT SiteWise asset hierarchy based on our industrial facility's structure. This process involves three main steps:

1. Defining the AWS IoT SiteWise Asset Generation Tool for Amazon Bedrock: We'll create a  tool that specifies the structure and requirements for generating AWS IoT SiteWise Hierarhcy assets.

2. Retrieving Context from the TIA Project: We'll use the extracted information from the TIA project, providing necessary context for asset generation.

3. Prompting Bedrock to generate the AWS IoT Assets Schema: Using the defined tool and retrieved context, we'll prompt Amazon Bedrock to generate a comprehensive AWS IoT SiteWise asset hierarchy schema.



### 3.1. Defining the AWS IoT SiteWise Asset Hierarchy Tool for Amazon Bedrock

Now, we will perform the same procedure to create a hierarchy of assets in AWS IoT Sitewise. As in the previous chapter, we will need to define the Amazon Bedrock Tool for the hierarchy:

In [ ]:
child_asset_eIDs = [asset["assetExternalId"] for asset in assets_schema["assets"]]
child_asset_names = [asset["assetName"] for asset in assets_schema["assets"]]

tool_object_sitewise_hierarchy = [
    {
        "toolSpec": {
            "name": "sitewise_assets_hierarchy",
            "description": "Create AWS IoT Sitewise Assets Hierarchy",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "assets": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "assetExternalId": {
                                        "type": "string",
                                        "description": "The eID_ of the parent asset",
                                        "enum": [
                                            "eID_Forming_Unit",
                                            "eID_Component_Fabrication_Unit"
                                        ]
                                    },
                                    "assetName": {
                                        "type": "string",
                                        "description": "the asset name of the parent asset",
                                        "enum": [
                                            "Forming Unit",
                                            "Component Fabrication Unit"
                                        ]
                                    },                                    
                                    "assetModelExternalId": {
                                        "type": "string",
                                        "description": "the appropriate model eID_ for the parent asset",
                                        "enum": [
                                            "eID_Production_Unit"
                                        ]
                                    },
                                    "assetHierarchies": {
                                        "type": "array",
                                        "description": "List of hierarchies that define relationships between parent and child assets",
                                        "items": {
                                            "type": "object",
                                            "properties": {
                                                "externalId": {
                                                    "type": "string",
                                                    "description": "The external ID defining the relationship between parent and child assets",
                                                    "enum": ["eID_Production-Cell"]
                                                },
                                                "childAssetExternalId": {
                                                    "type": "string",
                                                    "description": "The external ID of the child asset eID_ in the hierarchy",
                                                    "enum": child_asset_eIDs
                                                }
                                            },
                                            "required": [
                                                "externalId",
                                                "childAssetExternalId"
                                            ]
                                        }
                                    }
                                },
                                "required": [
                                    "assetExternalId",
                                    "assetName",
                                    "assetModelExternalId"
                                ]
                            }
                        }
                    },
                    "required": [
                        "assets"
                    ]
                }
            }
        }
    }
]

### 3.2 Prompt Amazon Bedrock to generate the AWS IoT Sitewise Assets Hierarchy Schema

Now that we have our contextual data, we'll use Amazon Bedrock to automatically generate the AWS IoT SiteWise asset hierarchy schema. 
This step will use generative AI to create a structured representation of our industrial assets and their relationships.

In [ ]:
# The exported tia_meta_data serve as context
context = tia_meta_data

# Define prompt for Amazon Bedrock to generate AWS IoT SiteWise asset JSON schema
prompt = """Please use the sitewise_assets_hierarchy tool to generate a JSON schema 
to assets hierarchy between parents and child assets for all cells for meta-data-bulk-import 
in AWS IoT Sitewise based on the content within the <content> tags."""

# Call Amazon Bedrock with prompt and metadata to generate AWS IoT SiteWise Assets Hierarchy Schema
hierarchy_schema = call_bedrock (prompt, context, tool_object_sitewise_hierarchy)

# Pretty print the generated schema with proper JSON formatting
print(json.dumps(hierarchy_schema, indent=4))

In [ ]:
# Visualize the JSON schema
visualize_json(hierarchy_schema)

In [ ]:
#The schema structure must always include asset models dictionary.
asset_models = {"assetModels": []}
modified_schema = {**asset_models, **hierarchy_schema}

#Write the assets schema to a JSON file
with open("genai_sitewise_hierarchy_schema.json", "w") as file:
    json.dump(modified_schema, file, indent=4)

## 4. Create SFC Sitewise target adapter config files

In this section, we'll use Amazon Bedrock to generate Shop Floor Connectivity (SFC) configurations for AWS IoT SiteWise.\
SFC is a config-only framework that facilitates data ingestion from industrial equipment to various AWS services.

SFC requires two main types of configurations:
1. Source configurations: Define how to collect data from industrial equipment (e.g., protocols like SIEMENS S7, Ethernet/IP, MQTT)
2. Target configurations: Specify how to send data to AWS services (e.g., AWS IoT SiteWise, Amazon Timestream, Amazon S3)

We'll focus on:
1. Defining a tool for generating SFC SiteWise target adapter configs, as specified in the [SFC Sitewise Target Config Schema Specification](https://github.com/aws-samples/shopfloor-connectivity/blob/mainline/docs/targets/aws-sitewise.md#awssitewisetargetconfiguration).
2. Using Amazon Bedrock to create the SFC target schema.

This process will automate the creation of configuration files essential for data collection and transmission to AWS IoT SiteWise.

### 4.1 Tool to generate SFC Sitewise target adapter config

As before, we will need to define a tool to provide as input to the Amazon Bedrock API. But first we'll extract all the data paths that SFC has discovered from our TIA Portal project. 
These data paths represent the specific points of data collection within our industrial system. This information is then expected as input to the definition of the tool object.

The following code reads a previously generated AWS IoT SiteWise target configuration file and extracts the DataPath for each property of every asset:

In [ ]:
# Extracting all data paths dicovered by SFC from TIA portal project

with open("include_generated_swtarget.json", 'r') as file:
    data = json.load(file)
    data_paths = [prop["DataPath"] for asset in data["Assets"] for prop in asset["Properties"]]          
#   print(data_paths)

We will feed this data to Amazon Bedrock along with other contetual data like PLC project meta data, and available AWS IoT SiteWise asset models

In [ ]:
asset_property_names = ["Current_Step", "Input_Buffer","Output_Buffer", "Seq_Running","Product_ID", "VIN", "Seq_Timer"]

tool_object_sfc = [
    {
        "toolSpec": {
            "name": "SFC_SWTarget_tool",
            "description": "Create SFC Configuration schema for AWS IoT SiteWise target adapter for data flow to AWS IoT SiteWise",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "Assets": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "AssetExternalId": {
                                        "type": "string",
                                        "description": "The eID_ of the sitewise assets",
                                        "enum": child_asset_eIDs
                                            },
                                    "Properties": {
                                        "type": "array",
                                        "description": "Measurements in a sitewise asset and associated datastreams ",
                                        "items": {
                                            "type": "object",
                                            "properties": {
                                                "DataPath": {
                                                    "type": "string",
                                                    "description": "The data stream that connects to sitewise asset properties" ,
                                                    "enum": data_paths
                                                },
                                                "PropertyName": {
                                                    "type": "string",
                                                    "description": "only the configured measurements or asset properties in the sitewise asset",
                                                    "enum": ["Current_Step", "Input_Buffer","Output_Buffer", "Seq_Running","Product_ID", "VIN", "Seq_Timer"]          
                                                }
                                            },
                                            "required": [
                                                "PropertyName"
                                            ]
                                        }
                                    }
                                },
                                "required": [
                                    "assetExternalId"
                                ]
                            }
                        },
                        "Region": {
                            "type": "string",
                            "default": f"{aws_region}"
                        },
                        "TargetType": {
                            "type": "string",
                            "default": "AWS-SITEWISE"
                        }
                    },
                    "required": [
                        "Assets",
                        "Region",
                        "TargetType"
                    ]
                }
            }
        }
    },
]


### 4.2 Prompt Amazon Bedrock to generate the SFC Target Schema

In this step, we'll leverage Amazon Bedrock to automatically match the extracted data paths with the appropriate AWS IoT SiteWise properties and generate a complete SFC target configuration in JSON format. This process automates the complex task of mapping industrial data points to their corresponding cloud-based asset properties.



In [ ]:
context = tia_meta_data + """
VIN : 13_VIN
13_Seq_Start is same as Sequence_Running
Seq_Complete:sequence isnot active
"""           

# Define prompt for Amazon Bedrock to generate SFC SiteWise Target Configuration JSON schema
prompt = """Please use the SFC_SWTarget_tool to generate the JSON schema for Shopfloor-Connectivity tool
config file for Sitewise target. For every sitewise asset and for every asset property described in the tool
map a data stream path to it based on the content within the <content> tags.""" 

# Call Amazon Bedrock with prompt and metadata to generate the SFC SiteWise Target Configuration JSON schema
sfc_sw_schema = call_bedrock(prompt,context, tool_object_sfc)

print(json.dumps(sfc_sw_schema, indent=4))

In [ ]:
# Write the AWS IoT SiteWise schema configuration to a JSON file with proper formatting and UTF-8 encoding
with open('genai_sfc_sitewise_target_conf.json', 'w', encoding='utf-8') as f:
    json.dump(sfc_sw_schema, f, ensure_ascii=False, indent=4)

In [ ]:
# Visualize the JSON schema
visualize_json(sfc_sw_schema)

<div>
<span style="font-size: 2em;">🎉</span><b> Congratulations!</b> You have now successfully generated AWS IoT SiteWise asset definitions and Shop Floor Connectivity (SFC) configurations using Amazon Bedrock's Generative AI capabilities. You should now have three files generated after executing this workshop. 

<span style="padding-left: 2em;">&#x2611; Make sure you can locate these files the root folder before moving ahead.</span>

<ol style="padding-left: 5em;">
    <li><span style="font-family: monospace;">genai_sitewise_assets_schema.json</span></li>
    <li><span style="font-family: monospace;">genai_sitewise_hierarchy_schema.json</span></li>
    <li><span style="font-family: monospace;">genai_sfc_sitewise_target_conf.json</span></li>
</ol>

<p align="right">
        ⏭️<em> Now let's continue with Notebook 03 to learn how to use these files <em>
    </p></div>

<div class="alert alert-block alert-info padding-left: 2em;">
<span style="font-size: 1.5em;">💡</span> Try experimenting with the prompts by changing the wording, adding more context, or adjusting parameters to observe how the responses differ each time. This exploration will help you understand the flexibility and power of generative AI in industrial asset management.
</div>